# Housekeeping
To make sure this is going to work, let's do some Python admin-ing. If you are on Mac, you can open a terminal window and run the following commands in order to open this notebook.

* conda create -n domopalooza19 python=3 jupyter pandas
* conda activate domopalooza19
* pip install pydomo requests jsonpickle
* jupyter notebook <filename>

## Setup

In [ ]:
from pydomo import Domo
import pandas as pd

pd.set_option('display.max_columns', 900)

creds = pd.read_csv('creds')
domo = Domo(creds['client_id'][0],creds['client_secret'][0])

## Create a new batch of users based off a CSV file
Here we have a CSV file provided by another business unit of users that need to be created in Domo. To begin with, we might want to list all of the users and check to see if there are any duplicates. The API will throw an error if you attempt to create two users with the same email address. 

Note that you can batch upload/modify users via the Domo UI (will show this during the demo).

Here we'll create one user just to pratice how it works.

In [ ]:
from pydomo.users import CreateUserRequest
from random import randint

new_user_test = CreateUserRequest()
new_user_test.name = 'This is a test'
new_user_test.email = "bogus_email{}@eemail.com".format(randint(0, 10000))
new_user_test.role = 'Privileged'
send_invite = False

nn_user = domo.users.create(new_user_test,send_invite)

du = pd.DataFrame(domo.users.list(500,0))
print(du)

### Create users from a list
Now we'll write a user-defined function that takes a Pandas DataFrame as an input and creates a user for each row provided.

In [ ]:
new_user_list = pd.read_csv('test_new_user_list.csv')

def add_user_list(ul,domo_object,invite=False):
    ur = CreateUserRequest()
    users_created = []
    for index, row in ul.iterrows():
        ur.name = row['displayName']
        ur.email = row['email']
        ur.role = row['role']
        users_created.append(domo_object.users.create(ur,invite))
    return(users_created)

created_users = add_user_list(new_user_list,domo)
all_users = domo.users.list(500,0)
cu_df = pd.DataFrame(all_users)
print(cu_df)

### Delete users
What if we want to delete some of these users? For example, it can be helpful to clear out all of the users I just created (in case I need to re-work this script at all). For now we'll leave it commented out.

In [ ]:
def delete_users(domo_object,keep_users=['jeremy.morris@domo.com','support@domo.com']):
    curr_users = domo_object.users.list(500,0)
    for x in curr_users:
        if x['email'] not in keep_users:
            domo_object.users.delete(x['id'])
    return 'done'
delete_users(domo)

If you have more than 500 users, you'll need to page through the results until you have retrieved the entire user list. The function below shows how that can be done.

In [ ]:
def get_all_users(domo_obj,batch_size=10):
    offset = 0
    all_current_users = []
    num_returned = batch_size
    while num_returned == batch_size:
        this_users = domo_obj.users.list(batch_size,offset)
        all_current_users.extend(this_users)
        offset += batch_size
        num_returned = len(this_users)
    return(all_current_users)

all_users = get_all_users(domo)
all_users

## Permission map

Have you ever wanted to generate a view of who can see pages and cards on those pages? The public API allows you to do the following.

* Query a list of all pages
* Retrieve basic information about that page including a list of all collections, cards, users and groups.
* Get basic information about individual collections of cards.

In [ ]:
def format_page_info(domo_obj,page_id):
    page_info = domo_obj.pages.get(page_id)
    
    info_keys = ['id','ownerId','name','locked']
    info_dict = {key:page_info[key] for key in info_keys}
    info_dict['tojoin'] = 1
    info_df = pd.DataFrame(info_dict,index=[0])
    
    cards = pd.DataFrame({'card_id':page_info['cardIds'],'tojoin':[1]*len(page_info['cardIds'])})
    
    info_cards = pd.merge(info_df,cards,on='tojoin')
    
    collections = domo_obj.pages.get_collections(page_id)
    collections_df = pd.DataFrame()
    collection_keys = ['id','title','description']
    for x in collections:
        collection_info = {key:x[key] for key in collection_keys}
        collection_info['tojoin'] = 1
        collection_info_df = pd.DataFrame(collection_info,index=[0])
        collection_card_df = pd.DataFrame({'card_id':x['cardIds'],'tojoin':[1]*len(x['cardIds'])})
        collection_add = pd.merge(collection_info_df,collection_card_df,on='tojoin')
        collections_df = pd.concat([collections_df,collection_add],sort=True)
    
    if len(collections) > 0:
        collection_cols = {'id':'collection_id','title':'collection_title','description':'collection_desc'}
        collections_df.rename(index=str,columns=collection_cols,inplace=True)
        collections_info = pd.merge(info_df,collections_df,on='tojoin')
        info_cards = pd.concat([info_cards,collections_info],sort=True)
    
    user_ids = page_info['visibility']['userIds']
    users = pd.DataFrame()
    if len(user_ids) > 0:
        users = pd.DataFrame({'user_id':user_ids,'tojoin':[1]*len(user_ids)})
        
    group_ids = page_info['visibility']['groupIds']
    groups = pd.DataFrame()
    if len(group_ids) > 0:
        for x in group_ids:
            uu = domo_obj.groups.list_users(x)
            tdf = pd.DataFrame({'group_id':[x]*len(uu),'user_id':uu,'tojoin':[1]*len(uu)})
            groups = pd.concat([groups,tdf],sort=True)
    
    visibility = pd.concat([users,groups],sort=True)
    
    out = pd.merge(info_cards,visibility,on='tojoin')
    final_name_changes = {'id':'page_id','locked':'page_locked','name':'page_name','ownerId':'page_owner'}
    out.drop(columns=['tojoin'],inplace=True)
    out.rename(index=str,columns=final_name_changes,inplace=True)
    
    
    return(out)

Now that we have this function, we can get a list of all pages in the instance and run the function for each page, concatenating all of the pages together.

In [ ]:
all_page_permissions = pd.DataFrame()
for x in list(domo.pages.list()):
    all_page_permissions = pd.concat([all_page_permissions,format_page_info(domo,x['id'])],sort=True)

all_page_permissions.reset_index(inplace=True,drop=True)
all_page_permissions.to_csv('permissions.csv',index=False)

## PDP Management
Have you ever wanted to do the following?

* Create a batch of PDP policies based on a data set?
* Copy PDP policies from one data set to another?
* Create a list of PDP policies and store that list in a data set?

### Create a batch of policies

For simplicity sake, we'll run through the entire user list creating one policy for every user against a specific data set. We'll assume that the user's email address is in a column of the data set. The function below will build a single policy allowing for additional fixed filters to be passed in as I have found this to be a useful feature. That function would then be used in a loop to add the policies to a dataset.

In [ ]:
from pydomo.datasets import DataSetRequest, Schema, Column, ColumnType, Policy
from pydomo.datasets import PolicyFilter, FilterOperator, PolicyType, Sorting

def email_policy_add(user_info,email_column='pdp_email',addtl_filters=[]):
    pdp_filter = PolicyFilter()
    pdp_filter.column = email_column  
    pdp_filter.operator = FilterOperator.EQUALS
    pdp_filter.values = [user_info['email']]
    
    all_filters = [pdp_filter]
    all_filters.extend(addtl_filters)
    
    pdp_request = Policy()
    pdp_request.name = "{} | {}".format(email_column,user_info['email'])
    pdp_request.filters = all_filters
    pdp_request.type = PolicyType.USER
    pdp_request.users = [user_info['id']]
    pdp_request.groups = []
    
    return(pdp_request)

Now that we have the function created, we can run this through a loop to add the policies to a data set. Note that I created a separate filter to add to each policy. This is what I do when I want to control the overall visibility of a row of data. If I need to hide it, I can flip the value to 0 in the data set hiding that row of data w/o needing to modify policies again. (Note that this allows you to add policies where values don't exist in the data.)

In [ ]:
extra_filter = PolicyFilter()
extra_filter.column = 'scorecard_instance'
extra_filter.operator = FilterOperator.EQUALS
extra_filter.values = ['1']

my_dataset = '91ab79cb-f018-4479-a8cb-909d445dbecb'

all_current_users = get_all_users(domo)

new_policies = []
for x in all_current_users:
    x_policy = email_policy_add(x,addtl_filters=[extra_filter])
    new_policies.append(domo.datasets.create_pdp(my_dataset,x_policy))
    
# Turn PDP on
ds_info = domo.datasets.update(my_dataset,{'pdpEnabled':True})

Now, we might want to remove all of the policies on a data set. To do that, we can request a list of all the policies and then delete each one in turn.

In [ ]:
all_pdp = domo.datasets.list_pdps(my_dataset)
for x in all_pdp:
    if x['name'] != 'All Rows' :
        domo.datasets.delete_pdp(my_dataset,x['id'])

# Turn PDP off
ds_info = domo.datasets.update(my_dataset,{'pdpEnabled':False})

### Copy policies from one data set to another
The simplest way to do this is to pull all of the policies from one data set, and run a loop to create them on another. In this example, I'll copy policies from the data set above to two other data sets.

In [ ]:
ds_new_pdp = ['c4416a0c-d7a0-4a6d-a37f-eeed7bdaf530',
              'ad84816d-68bb-4aae-b4c3-8c83ac4f6359']

policies_to_copy = domo.datasets.list_pdps(my_dataset)

copied_policies = {}

for x in ds_new_pdp:
    copied_policies[x] = []
    for y in policies_to_copy:
        if y['name'] != 'All Rows':
            np = domo.datasets.create_pdp(x,y)
            copied_policies[x].append(np)

copied_policies

### Copy policies into a Domo dataset
This can be done by calling the list_pdps function and then formatting the output into a DataFrame and then uploading that DataFrame to Domo. We'll do this for three data sets.

In [ ]:
pdp_ds = ds_new_pdp
pdp_ds.append(my_dataset)

def format_policies(domo_obj,ds):
    a = domo_obj.datasets.list_pdps(ds)
    out = pd.DataFrame()
    for x in a:
        tt = pd.DataFrame({'ds_id':ds,'pdp_id':x['id'],'pdp_name':x['name'],
                           'n_users':len(x['users']),'n_groups':len(x['groups'])},index=[0])
        out = pd.concat([out,tt])
    out.reset_index(inplace=True,drop=True)
    return out

all_my_policies = pd.DataFrame()
for x in pdp_ds:
    tt = format_policies(domo,x)
    all_my_policies = pd.concat([all_my_policies,tt])

all_my_policies.reset_index(inplace=True,drop=True)

In [ ]:
from pydomo.datasets import DataSetRequest, Schema, Column, ColumnType, Policy
from pydomo.datasets import PolicyFilter, FilterOperator, PolicyType, Sorting

new_ds = DataSetRequest()
new_ds.name = 'PDP List'
new_ds.description = 'Downloaded via API'
new_ds.schema = Schema([Column(ColumnType.STRING, 'ds_id'),
                        Column(ColumnType.DOUBLE, 'policy_id'),
                       Column(ColumnType.STRING, 'policy_name'),
                       Column(ColumnType.DOUBLE, 'n_users'),
                       Column(ColumnType.DOUBLE, 'n_groups')])

dataset = domo.datasets.create(new_ds)

Now that the data set has been created, we need to upload data. This can be done by converting the data set to a CSV string and using the `data_import` function.

In [ ]:
my_ds_str = all_my_policies.to_csv(index=False)
domo.datasets.data_import(dataset['id'], my_ds_str)

This can be repeated to replace all of the data.

In [ ]:
my_ds_str = all_my_policies.to_csv(index=False)
domo.datasets.data_import(dataset['id'], my_ds_str)